[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab2/lab2-parte2.ipynb)

# Práctica 1: Redes neuronales desde cero con TensorFlow - Parte 2

En esta segunda parte de la práctica vamos a utilizar TensorFlow para implementar y entrenar la misma red neuronal que desarrollamos con Numpy en la parte 1.

Necesitaremos, por tanto, la librería TensorFlow además de las ya utilizadas Numpy y tensorflow_datasets.

In [7]:
# COLAB - Para ejecutar desde Colab, descomentar la siguiente línea
%tensorflow_version 2.x
# LOCAL - Para ejecutar en Local, descomentar la siguiente línea
#!pip3 install tensorflow numpy tensorflow-datasets

import tensorflow as tf
import numpy as np
import tensorflow_datasets as tfds

# Establecemos una semilla aleatoria para que los resultados sean reproducibles en distintas ejecuciones
np.random.seed(1234567)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


Cargaremos el conjunto de datos `german_credit_numeric` del que tomaremos el primer lote de 100 elementos, tal como hicimos en la parte 1 de la práctica. Obtendremos dos tensores (`vectores_x` y `etiquetas`) que serán los que utilizaremos posteriormente.

In [8]:
import tensorflow_datasets as tfds

# Cargamos el conjunto de datos
ds = tfds.load('german_credit_numeric', split='train')

tamano_lote = 100

elems = ds.batch(tamano_lote)
lote_entrenamiento = None
for elem in elems:
    lote_entrenamiento = elem
    break

vectores_x = tf.cast(lote_entrenamiento["features"], dtype=tf.float64)
etiquetas = tf.cast(lote_entrenamiento["label"], dtype=tf.float64)

## Declaración del modelo

En primer lugar, debemos crear en TensorFlow el grafo de operaciones que representa nuestro modelo. Para ello:
 1. Creamos las variables que TF optimizará, es decir, los parámetros del modelo.
 1. Creamos el grafo de operaciones que producen la predicción a partir de la entrada y las variables. En este caso utilizaremos funciones que relacionen variables de TF con tensores que contendrán datos utilizando operaciones de TF.

In [9]:
# Variables auxiliares
tamano_entrada = 24
h0_size = 5
h1_size = 3

# CREACIÓN DE LAS VARIABLES
W0 = tf.Variable(np.random.randn(h0_size, tamano_entrada), dtype=tf.float64, name='W0')
b0 = tf.Variable(np.random.randn(h0_size, 1), dtype=tf.float64, name='b0')

W1 = tf.Variable(np.random.randn(h1_size, h0_size), dtype=tf.float64, name='W1')
b1 = tf.Variable(np.random.randn(h1_size, 1), dtype=tf.float64, name='b1')

W2 = tf.Variable(np.random.randn(1, h1_size), dtype=tf.float64, name='W2')
b2 = tf.Variable(np.random.randn(1, 1), dtype=tf.float64, name='b2')

VARIABLES = [W0, b0, W1, b1, W2, b2]

# CREACIÓN DEL GRAFO DE OPERACIONES
@tf.function
def sigmoide(x):
    return 1 / (1 + tf.exp(-x))

@tf.function
def capa_sigmoide(x, W, b):
    # b se transpone para que sea (1, n_out) y se pueda sumar por broadcasting
    return sigmoide(tf.matmul(x, tf.transpose(W)) + tf.transpose(b))

@tf.function
def predice(x):
    h0 = capa_sigmoide(x, W0, b0)
    h1 = capa_sigmoide(h0, W1, b1)
    y = capa_sigmoide(h1, W2, b2)
    return y


# Verificación
x_test = np.random.randn(1,tamano_entrada)
y_pred = predice(x_test)
print(y_pred)
np.testing.assert_almost_equal(0.48001507, y_pred.numpy(), err_msg='Revisa tu implementación')

tf.Tensor([[0.48001507]], shape=(1, 1), dtype=float64)


## Entrenamiento del modelo
El modelo declarado ya se puede utilizar para hacer predicciones pasándole a la función `predice` un tensor con datos (tal como se ha hecho en el apartado de verificación de la celda anterior). Sin embargo, como vimos en la parte 1, este modelo no está ajustado a los datos de entrada, por lo que producirá malas predicciones.

Debemos encontrar un conjunto de valores para los parámetros ($\mathbf{W}_2$, $b_2$, $\mathbf{W}_1$, $\mathbf{b}_1$, $\mathbf{W}_0$ y $\mathbf{b}_0$) que minimicen la función de coste. TensorFlow nos ayuda a optimizar este proceso.

TensorFlow permite configurar el proceso de optimización, por lo que deberemos indicarle:
 1. Qué función de pérdida queremos. En nuestro caso habíamos elegido la entropía cruzada binaria.
 1. Qué método de optimización utilizar. Como en la parte 1, utilizaremos descenso de gradiente.

Por el momento crearemos sendas variables para almacenar ambas configuraciones. Al estar organizado de esta manera, utilizar una función de pérdida distinta o un algoritmo de optimización diferente será tan sencillo como cambiar estas variables.

In [10]:
fn_perdida = tf.keras.losses.BinaryCrossentropy()

optimizador = tf.keras.optimizers.SGD(learning_rate=0.01)
#optimizador = tf.keras.optimizers.Adam(0.001)

### El bucle de entrenamiento

El bucle de entrenamiento será análogo al utilizado en la parte 1. Consistirá en ejecutar un número preestablecido (`NUM_EPOCHS`) de pasos de entrenamiento. En cada paso haremos lo siguiente:
 1. Tomar los datos de entrada y calcular las predicciones que hace el modelo en su estado actual
 1. Calcular el coste (la media de las pérdidas de cada predicción)
 1. Utilizar el valor de coste para actualizar cada variable en dirección de su gradiente

Crearemos una función `paso_entrenamiento` que realice este trabajo. TensorFlow se ocupará de calcular los gradientes y realizar las actualizaciones de las variables. Para calcular los gradientes, TensorFlow utiliza un `GradientTape`. Todas las operaciones con tensores que se realicen dentro del entorno en que está declarado este `GradientTape` quedarán registradas y eso nos permitirá obtener los gradientes directamente del `GradientTape` con una simple llamada. Puedes comprobar su funcionamiento en el ejemplo.



In [11]:
@tf.function
def paso_entrenamiento(x, y):
    # Declaración del GradientTape que registrará las operaciones
    with tf.GradientTape() as tape:
        # TODO - Completa la siguiente línea para que calcule las predicciones
        y_pred = predice(x) # Forward pass

        # Cálculo de la pérdida utilizando la función que hemos escogido anteriormente
        perdida = fn_perdida(y, y_pred)

        # Consultar los gradientes es tan sencillo como indicarle dos cosas:
        #    1. la función cuyo gradiente queremos obtener
        #    2. la lista de variables respecto a las cuales queremos calcular el gradiente
        # La función nos devolverá una lista con el gradiente correspondiente a cada variable de la lista
        gradientes = tape.gradient(perdida, VARIABLES)

        # Realizar la actualización de las variables solo requiere esta llamada. Se le pasa una lista de tuplas (gradiente, variable)
        optimizador.apply_gradients(zip(gradientes, VARIABLES))

        # Para poder mostrar la tasa de acierto, la calculamos a cada paso
        fallos = tf.abs(tf.reshape(y,(tamano_lote, 1)) - y_pred)
        tasa_acierto = tf.reduce_sum(1 - fallos)

        # Devolvemos estos dos valores para poder mostrarlos por pantalla cuando estimemos conveniente
        return (perdida, tasa_acierto)



# PROCESO DE ENTRENAMIENTO
num_epochs = 10000
for epoch in range(num_epochs):
    perdida, tasa_error = paso_entrenamiento(vectores_x, etiquetas)

    if epoch % 100 == 99:
        print("Epoch:", epoch, 'Pérdida:', perdida.numpy(), 'Tasa de acierto:', tasa_error.numpy()/tamano_lote)


Epoch: 99 Pérdida: 0.65055674 Tasa de acierto: 0.5237970133452802
Epoch: 199 Pérdida: 0.6124922 Tasa de acierto: 0.5523350431790992
Epoch: 299 Pérdida: 0.5955333 Tasa de acierto: 0.5707902872738352
Epoch: 399 Pérdida: 0.58786637 Tasa de acierto: 0.5827775341930248
Epoch: 499 Pérdida: 0.5843308 Tasa de acierto: 0.5906537392653814
Epoch: 599 Pérdida: 0.58265615 Tasa de acierto: 0.5958912321056649
Epoch: 699 Pérdida: 0.58182967 Tasa de acierto: 0.5994109381188322
Epoch: 799 Pérdida: 0.5813935 Tasa de acierto: 0.6017976062886653
Epoch: 899 Pérdida: 0.5811384 Tasa de acierto: 0.6034289972774414
Epoch: 999 Pérdida: 0.58096784 Tasa de acierto: 0.6045529112807128
Epoch: 1099 Pérdida: 0.5808373 Tasa de acierto: 0.6053338820409074
Epoch: 1199 Pérdida: 0.5807259 Tasa de acierto: 0.6058821511222175
Epoch: 1299 Pérdida: 0.58062387 Tasa de acierto: 0.6062720693953498
Epoch: 1399 Pérdida: 0.58052665 Tasa de acierto: 0.6065540121017616
Epoch: 1499 Pérdida: 0.580432 Tasa de acierto: 0.6067622191067037


El uso de TensorFlow nos ha permitido abstraernos de los detalles de implementación y del cálculo de derivadas para centrarnos en la arquitectura de nuestro modelo.